In [81]:
import pandas as pd
import pyarrow.parquet as pq
import os
import airportsdata 
from geopy.distance import geodesic

In [4]:
airports = airportsdata.load()

dataset = pq.ParquetDataset('c:\\Users\\lukas\\OneDrive - Imperial College London\\0_Sustainable Transport\\2021_COP26',
                            ) # filters=[('columnName', 'in', filterList)]
df = dataset.read(use_threads=True).to_pandas()

In [151]:
delegates_path = os.getcwd() + '\\origin_airports_delegates.csv'
data_delegates = pd.read_csv(delegates_path)

iata_icao_path = os.getcwd() + '\\iata-icao.csv'
data_iata_icao = pd.read_csv(iata_icao_path)

In [152]:
airport_location = data_delegates.merge(data_iata_icao, on='iata', how='left')[['iata','airport','latitude','longitude']]
airport_location

,iata,airport,latitude,longitude
0,AUH,Abu Dhabi International Airport,24.43300,54.65110
1,GRU,NaN,23.42620,46.48000
2,PEK,Beijing Capital International Airport,40.07250,116.59800
3,LOS,Murtala Muhammed International Airport,6.57737,3.32116
4,CGK,Soekarno–Hatta International Airport,-6.12556,106.65600
...,...,...,...,...
205,ZRH,Zurich Airport,47.46470,8.54917
206,KIV,Chisinau International Airport,46.92770,28.93100
207,ASM,Asmara International Airport,15.29190,38.91070
208,MGA,Augusto C. Sandino International Airport,12.14150,-86.16820


In [153]:
# Check for NAN values 
# airport_location.isna()
# airport_location = airport_location.dropna()#isna().sum()#.any(axis=1).count
# airport_location[airport_location.isna().any(axis=1)]

,iata,airport,latitude,longitude
1,GRU,NaN,23.4262,46.4800
10,JFK,NaN,40.6446,73.7797


In [154]:
dubai_airport = pd.DataFrame({'latitude_A': [airports.get('OMDB')['lat']],
                     'longitude_A': [airports.get('OMDB')['lon']]})

delegate_airports = airport_location[['latitude','longitude']]

# Function to calculate distance using geopy.distance
def calculate_distance(row):
    coord_A = (dubai_airport['latitude_A'].iloc[0], dubai_airport['longitude_A'].iloc[0])
    coord_B = (row['latitude'], row['longitude'])
    return geodesic(coord_A, coord_B).kilometers

airport_location['distance_to_dubai'] = delegate_airports.apply(calculate_distance,axis=1)

airport_location

,iata,airport,latitude,longitude,distance_to_dubai
0,AUH,Abu Dhabi International Airport,24.43300,54.65110,115.950398
1,GRU,NaN,23.42620,46.48000,923.827135
2,PEK,Beijing Capital International Airport,40.07250,116.59800,5856.254969
3,LOS,Murtala Muhammed International Airport,6.57737,3.32116,5899.623220
4,CGK,Soekarno–Hatta International Airport,-6.12556,106.65600,6543.695620
...,...,...,...,...,...
205,ZRH,Zurich Airport,47.46470,8.54917,4773.283197
206,KIV,Chisinau International Airport,46.92770,28.93100,3352.541593
207,ASM,Asmara International Airport,15.29190,38.91070,2039.020269
208,MGA,Augusto C. Sandino International Airport,12.14150,-86.16820,14143.820362


In [155]:
sum(airport_location['distance_to_dubai'])

1398796.6436137748

#### Heatmap

In [156]:
import folium
from folium.plugins import FastMarkerCluster
from folium.plugins import HeatMap

def Generate_basemap():
    basemap = folium.Map(location=[airports.get('OMDB')['lat'] , airports.get('OMDB')['lon']])
    return basemap

basemap = Generate_basemap()
FastMarkerCluster(airport_location[['latitude', 'longitude']]).add_to(basemap)
HeatMap(airport_location[['latitude', 'longitude']]).add_to(basemap)
basemap